# LightGlue Demo
In this notebook we match two pairs of images using LightGlue with early stopping and point pruning.

In [1]:
import sys
sys.path.insert(0,'/home/rmqlife/work/LightGlue')

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch

torch.set_grad_enabled(False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint", filter_threshold=0.9).eval().to(device)

/home/rmqlife/miniconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/rmqlife/miniconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Easy example
The top image shows the matches, while the bottom image shows the point pruning across layers. In this case, LightGlue prunes a few points with occlusions, but is able to stop the context aggregation after 4/9 layers.

In [2]:

import json
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import os

def replace_path(file_path, src, dst):
    directory, filename = os.path.split(file_path)  
    new_filename = filename.replace(src, dst)
    
    return os.path.join(directory, new_filename)
    
def replace_rgb_to_depth(file_path):
    return replace_path(file_path, 'rgb', 'depth')


# Factory function to create a pointcloud from an RGB-D image and a camera. Given depth value d at (u, v) image coordinate, the corresponding 3d point is:

# z = d / depth_scale
# x = (u - cx) * z / fx
# y = (v - cy) * z / fy
def project_to_3d(points, depth, intrinsics, show=True):
    if show:
        plt.imshow(depth)
    
    points_3d = list()
    
    for x,y in points:
        x = math.floor(x) 
        y = math.floor(y)
        d = depth[y][x]        
        # Plot points (x, y) on the image
        if show:
            if d>0:
                plt.scatter(x, y, color='blue', s=10)  # Adjust the size (s) as needed
            else:
                plt.scatter(x, y, color='red', s=10)

        # 3d point in meter
        z = d / 1000
        x = (x - intrinsics['cx']) * z / intrinsics['fx'] 
        y = (y - intrinsics['cy']) * z / intrinsics['fy'] 
        
        if show:
            print(f'x:{x} \t y:{y} \t z:{z}')
        points_3d.append((x,y,z))
        
    if show:
        plt.axis('off')  # Turn off axis labels
        plt.show()
    
    
    return points_3d
    
def load_intrinsics(json_file):
    with open(json_file, "r") as file:
        intrinsic_params = json.load(file)
    return intrinsic_params


def plot_matching(image0, image1, pts0, pts1):
    axes = viz2d.plot_images([image0, image1])
    viz2d.plot_matches(pts0, pts1, color="lime", lw=0.2)
    
def match_with_lightglue(image0, image1):
    feats0 = extractor.extract(image0.to(device))
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension

    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    return m_kpts0, m_kpts1


id1=26
id2=19

image_path1 = f"images-20240611-194515/rgb_{id1}.png"
image_path2 = f"images-20240611-194515/rgb_{id2}.png"
traj_path = f"images-20240611-194515/traj.npy"

image0 = load_image(image_path1)
image1 = load_image(image_path2)

pts0, pts1 = match_with_lightglue(image0, image1)
plot_matching(image0, image1, pts0, pts1)
plt.show()
print(pts0.shape)

# Example usage:
intrinsics = load_intrinsics("intrinsic_parameters.json")
print(intrinsics)

depth_path1 = replace_rgb_to_depth(image_path1)
depth_path2 = replace_rgb_to_depth(image_path2)

rgb1 = cv2.imread(image_path1)
depth1 = cv2.imread(depth_path1, cv2.IMREAD_UNCHANGED)

rgb2 = cv2.imread(image_path2)
depth2 = cv2.imread(depth_path2, cv2.IMREAD_UNCHANGED)

print(depth2.shape)
image_size = depth2.shape

pt3d1 = project_to_3d(pts0[:], depth1, intrinsics)
pt3d2 = project_to_3d(pts1[:], depth2, intrinsics)

print(pt3d1)
print(pt3d2)


FileNotFoundError: No image at path images-20240611-194515/rgb_26.png.

In [5]:

import cv2
import numpy as np
from lightglue_util import MyGlue, replace_rgb_to_depth, load_intrinsics, print_array
glue = MyGlue('LightGlue')

id1=26
id2=19

home = 'images-20240611-201439'

image_path1 = f"{home}/rgb_{id1}.png"
image_path2 = f"{home}/rgb_{id2}.png"
traj_path = f"{home}/traj.npy"
rgb1 = cv2.imread(image_path1)
rgb2 = cv2.imread(image_path2)

pts0, pts1 = glue.match_with_lightglue(rgb1, rgb2)

[ WARN:0@84.298] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images-20240611-201439/rgb_26.png'): can't open/read file: check file path/integrity
[ WARN:0@84.298] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images-20240611-201439/rgb_19.png'): can't open/read file: check file path/integrity


TypeError: MyGlue.match_with_lightglue() missing 1 required positional argument: 'verbose'

In [9]:
print(pts1[0])
pts0 = np.array(pts0)
pts1 = np.array(pts1)

# img_matches = cv2.drawMatches(rgb1, pts0, rgb2, pts1, None, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plot_matching(image0, image1, pts0, pts1)


[435  30]


NameError: name 'plot_matching' is not defined